# Práctica 4 - Neo4j
## Consultas

In [2]:
from neo4j import GraphDatabase
import pandas as pd

### Conexión

In [3]:
database_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "neo5j"))
session = database_connection.session()  

### Definiciones auxiliares

In [4]:
#Separando los resultados por variable

def split_variables(resobj):
    varlist = []
    keys = resobj.keys()
    data = resobj.data()
    for idx, var in enumerate(keys):
        #vardict = { keys[kidx] : var for kidx, var in enumerate(data) }
        attriblist = [dct[var] for dct in data]
        varlist.append(attriblist)
        
    vardict = { keys[kidx] : val for kidx, val in enumerate(varlist) }
    return vardict

In [175]:
# Prueba

c0 = 'match (n) RETURN n'

result = session.run(c0)
res = split_variables(result)
res['n'][:5]

[{'Id_Cliente': 'c0'},
 {'Id_Cliente': 'c1'},
 {'Id_Cliente': 'c2'},
 {'Id_Cliente': 'c3'},
 {'Id_Cliente': 'c4'}]

### Consultas

#### Consulta 1

Dado un estado y un producto, buscar tiendas donde se pueda encontrar.

In [179]:
c1 = "MATCH (p:productos {Tipo_Producto:'BRANDY'}) \
MATCH (e:estados {Nombre_Estado:'AGUASCALIENTES'}) \
MATCH (t:tiendas) \
MATCH (e) -[v]- (t) -- (p) \
RETURN t"

result = session.run(c1)
res = split_variables(result)
res['t'][:5]

[{'Id_Tienda': '137t', 'Nombre_Tienda': 'TIENDAS SORIANA  S.A. DE C.V.'},
 {'Id_Tienda': '82t',
  'Nombre_Tienda': 'SUSHITO - JUAN FERNANDO OROZCO CARDONA'},
 {'Id_Tienda': '74t', 'Nombre_Tienda': 'RESTAURANTE AUGUSTOÃ¯S S.A. DE C.V.'},
 {'Id_Tienda': '66t',
  'Nombre_Tienda': 'WINGS AIR FORCE- FRANQUICIAS WINGS AIR FORCE  S. DE R.L. DE C.V.'},
 {'Id_Tienda': '65t', 'Nombre_Tienda': 'LA MANSION- RICHYCARLS  S.A. DE C.V.'}]

In [180]:
def consulta1(producto,estado,num_resultados):
    s1 = "MATCH (p:productos {Tipo_Producto :'"+producto+"'}) "
    s2 = "MATCH (e:estados {Nombre_Estado:'"+estado+"'}) "
    s3 = "MATCH (t:tiendas) \
MATCH (e) -[v]- (t) -- (p) \
RETURN t"

    c1 = s1+s2+s3
    result = session.run(c1)
    res = split_variables(result)
    if len(res['t']) > num_resultados:
        #print(res['t'][:num_resultados])
        return list(map((lambda dct : dct['Nombre_Tienda']), res['t'][:num_resultados]))
    else:
        #print(res['t'])
        return list(map((lambda dct : dct['Nombre_Tienda']), res['t']))   

In [213]:
producto = 'BRANDY'
estado = 'AGUASCALIENTES'

out = consulta1(producto, estado, 5)

print(f"Tiendas donde se puede encontrar {producto} en {estado}: \n")

for elem in out:
    print(elem)

Tiendas donde se puede encontrar BRANDY en AGUASCALIENTES: 

TIENDAS SORIANA  S.A. DE C.V.
SUSHITO - JUAN FERNANDO OROZCO CARDONA
RESTAURANTE AUGUSTOÃ¯S S.A. DE C.V.
WINGS AIR FORCE- FRANQUICIAS WINGS AIR FORCE  S. DE R.L. DE C.V.
LA MANSION- RICHYCARLS  S.A. DE C.V.


#### Consulta 2

Dado un estado y una tienda, revisar si existen incumplimientos en sus productos.

In [186]:
c2 = "MATCH (e:estados {Nombre_Estado :'BAJA CALIFORNIA'}) \
MATCH (t:tiendas {Nombre_Tienda: 'CARNE MART SUCURSAL TECATE'}) \
MATCH (p:productos) \
MATCH (e) -- (t) -[v]- (p) \
RETURN e, t, p, v.incumplimiento"

result = session.run(c2)
res = split_variables(result)
res

{'e': [{'Nombre_Estado': 'BAJA CALIFORNIA', 'Id_Estado': 'BAJA CALIFORNIA'}],
 't': [{'Id_Tienda': '139t', 'Nombre_Tienda': 'CARNE MART SUCURSAL TECATE'}],
 'p': [{'Id_Producto': '43p', 'Tipo_Producto': 'PRODUCTOS ALIMENTICIOS'}],
 'v.incumplimiento': ['NO SE DETECTO INCUMPLIMIENTO']}

In [196]:
def consulta2(estado,tienda):
    s1 = "MATCH (e:estados {Nombre_Estado :'"+estado+"'}) "
    s2 = "MATCH (t:tiendas {Nombre_Tienda: '"+tienda+"'}) "
    s3 = "MATCH (p:productos) \
MATCH (e) -- (t) -[v]- (p) \
RETURN e, t, p, v.incumplimiento"
    c2 = s1+s2+s3

    result = session.run(c2)
    res = split_variables(result)
    prod = list(map(lambda dct : dct['Tipo_Producto'], res['p']))
    return prod, res['v.incumplimiento']

In [215]:
estado = 'BAJA CALIFORNIA'
tienda = 'CARNE MART SUCURSAL TECATE'

out = consulta2(estado, tienda)

print(f"Para la tienda {tienda} en el estado de {estado}:\n")

print(f"{'Producto':^40} | {'Incumplimientos':^30}")
for idx in range(len(out[0])):
    print(f"{out[0][idx]:<40} | {out[1][idx]:<30}")

Para la tienda CARNE MART SUCURSAL TECATE en el estado de BAJA CALIFORNIA:

                Producto                 |        Incumplimientos        
PRODUCTOS ALIMENTICIOS                   | NO SE DETECTO INCUMPLIMIENTO  


#### Consulta 3

Dado un estado y un producto, buscar alternativas que no tengan incumplimientos.

In [210]:
c3 = "MATCH (e:estados {Nombre_Estado :'BAJA CALIFORNIA'}) \
MATCH (p:productos {Tipo_Producto:'BRANDY'}) \
MATCH (t:tiendas) \
MATCH (e) -- (t) -[v {incumplimiento : 'NO SE DETECTO INCUMPLIMIENTO'}]- (p) \
RETURN e, p, t, v.incumplimiento"

result = session.run(c3)
res = split_variables(result)
res

{'e': [{'Nombre_Estado': 'BAJA CALIFORNIA', 'Id_Estado': 'BAJA CALIFORNIA'},
  {'Nombre_Estado': 'BAJA CALIFORNIA', 'Id_Estado': 'BAJA CALIFORNIA'}],
 'p': [{'Id_Producto': '2p', 'Tipo_Producto': 'BRANDY'},
  {'Id_Producto': '2p', 'Tipo_Producto': 'BRANDY'}],
 't': [{'Id_Tienda': '137t', 'Nombre_Tienda': 'TIENDAS SORIANA  S.A. DE C.V.'},
  {'Id_Tienda': '1692t', 'Nombre_Tienda': '7 ELEVEN MEXICO  S.A. DE C.V.'}],
 'v.incumplimiento': ['NO SE DETECTO INCUMPLIMIENTO',
  'NO SE DETECTO INCUMPLIMIENTO']}

In [216]:
def consulta3(estado,producto):
    s1 = "MATCH (e:estados {Nombre_Estado :'"+estado+"'}) "
    s2 = "MATCH (p:productos {Tipo_Producto:'"+producto+"'}) "
    s3 = "MATCH (t:tiendas) \
MATCH (e) -- (t) -[v {incumplimiento : 'NO SE DETECTO INCUMPLIMIENTO'}]- (p) \
RETURN e, p, t, v.incumplimiento"

    c3 = s1+s2+s3

    result = session.run(c3)
    res = split_variables(result)
    return list(map(lambda dct : dct['Nombre_Tienda'], res['t']))

In [219]:
estado = 'BAJA CALIFORNIA'
producto = 'BRANDY'

out = consulta3(estado, producto)

print(f'Tiendas en {estado} que no tienen incumplimientos del producto {producto}: \n')
for elem in out:
    print(elem)

Tiendas en BAJA CALIFORNIA que no tienen incumplimientos del producto BRANDY: 

TIENDAS SORIANA  S.A. DE C.V.
7 ELEVEN MEXICO  S.A. DE C.V.
